In [11]:
%reload_ext autoreload
%autoreload 0
%matplotlib inline
import torch
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification, TrainingArguments, Trainer
from fastai import *
from fastai.text.all import *
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx
from sklearn.model_selection import train_test_split
sys.path.append("../../..")
from utils import *
from torch.utils.data import Dataset


In [12]:
model_dir = "/home/sm1073/.cache/huggingface/hub/models--zhihan1996--DNABERT-2-117M/DNABERT-2-117M"
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

In [13]:
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True)

Some weights of BertModel were not initialized from the model checkpoint at /home/sm1073/.cache/huggingface/hub/models--zhihan1996--DNABERT-2-117M/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
path = Path('/home/sm1073/Documents/independent_project/')

In [15]:
df = pd.read_csv(path/'x_inactivation_genes.csv')
df.head()

,GeneName,Sequence,Label
0,ASMTL,AAGTGCGGACGCCCGGCTCCCGGCGTGGACGCCATGGTGCTGTGCCCGGTGATTGGGAAGCTGCTGCACAAGCGCGTGGTGCTGGCCAGCGCCTCCCCACGCCGTCAGGAGATCCTCAGCAACGCGGGTCTCAGGTTTGAGGTGGTCCCCTCCAAGTTTAAAGAGAAGCTGGACAAAGCCTCCTTCGCTACTCCGTATGGGTACGCCATGGAGACCGCCAAGCAGAAGGCCCTGGAGGTGGCCAACCGGCTGTACCAGAAAGACCTGCGGGCCCCCGACGTGGTCATTGGAGCGGACACGATCGTGACAGTCGGGGGGCTGATTCTGGAGAAGCCGGTGGACAAGCAGGACGCCTACAGGATGCTGTCCCGGTTGAGTGGGAGAGAACACAGCGTGTTCACAGGTGTCGCGATCGTCCACTGCTCCAGCAAAGACCATCAGCTGGACACCAGGGTCTCGGAATTCTACGAGGAAACGAAGGTGAAGTTCTCGGAGCTGTCCGAGGAGCTGCTCTGGGAATACGTCCACAGCGGGGAGCCCATGGACAAAGCTGGCGGCTACGGGATCCAGGCCCTGGGCGGCATGCTGGTGGAGTC...,1
1,RIBC1,CGGGCGACCGGCAAATGTCGCGAGAATACGTCCAGGCCTAACGGGATTTTCGCCTTCCGGCAGGAGTGGGACCAGGGGCGGGGACTGATGTTGCGTATACGAGGAGCGAGCTGACGACTGGCGTGGTGCCTCCGCCCCCCTTTCTTGTTGCATGGCAACAACAAACACTTGCTCCTGCGGACCAGATCCCCGGAGGAGTTGTTGCTGGGCGGAGTCAGAGCAGTTCTTGAGAGTTCAGAAACTGCCGCCGTAACCCAAGAACAAAGTTGCATGCAGAGCCAACCTAAGAGCTTCAAAATCGGGATTCCTTACTGAGAAAGATGTATAACATAAAACAGTCAACAGATACCAAGGAAGCAGCAGCCATCGAGGCTAGAAGAAATCGAGAAAAAGAGCGACAAAACCGATTCTTCAATGTGCGGAACCGAGTCATGGGGGTGGATGTCCAGGCCCTGAACAACCAGGTGGGAGACCGAAAGCGTCGGGAAGCAGCAGAAAGAAGCAAGGAGGCAGCTTATGGTACCAGCCAGGTGCAGTATGATGTGGTAGTCCAGATGTTAGAGAAGGAAGAGGCAGATCGAACACGTCAGCTGGCC...,1
2,MPC1L,CTGTGGCGGAAGATGAGAGATAACTTCCAGAGCAAGGAGTTCCGGGAATACGTGAGCAGCACTCACTTCTGGGGTCCCGCGTTCAGCTGGGGCCTTCCGCTGGCTGCCTTTAAAGACATGAAGGCGTCGCCGGAGATCATCAGTGGCCGCATGACAACAGCGCTCATCTTGTACTCGGCGATCTTCATGCGCTTCGCCTACCGCGTACAGCCTCGAAACCTGCTGCTGATGGCGTGCCACTGTACCAACGTGATGGCGCAGAGTGTGCAGGCCAGTC,1
3,DDX3X,CTTTCCCCTTACTCCGCTCCCCTCTTTTCCCTCCCTCTCCTCCCCTTCCCTCTGTTCTCTCCTCCTCTTCCCCTCCCCTCCCCCGTCCGGAGCACTCTATATTCAAGCCACCGTTTCCTGCTTCACAAAATGGCCACCGCACGCGACACCTACGGTCACGTGGCCTGCCGCCCTCTCAGTTTCGGGAATCTGCCTAGCTCCCACTAAGGGGAGGCTACCCGCGGAAGAGCGAGGGCAGATTAGACCGGAGAAATCCCACCACATCTCCAAGCCCGGGAACTGAGAGAGGAAGAAGAGTGAAGGCCAGTGTTAGGAAAAAAAAAACAAAAACAAAAAAAACGAAAAACGAAAGCTGAGTGCATAGAGTTGGAAAGGGGAGCGAATGCGTAAGGTTGGAAAGGGGGGCGAAGAGGCCTAGGTTAACATTTTCAGGCGTCTTAGCCGGTGGAAAGCGGGAGACGCAAGTTCTCGCGAGATCTCGAGAACTCCGAGGCTGAGACTAGGGTTTTAGCGGAGAGCACGGGAAGTGTAGCTCGAGAGAACTGGGACAGCATTTCGCACCCTAAGCTCCAAGGCAGGACTGCTAGGGGCGACAG...,1
4,HDHD1,CTCAGTGCGCGTGCGCGGGGCGGGCGGGTGCGCGCGCACTTCCTCCTCGCCCCCACCCAGACCCAGAAGGCGCCACCATGGCGGCGCCCCCGCAGCCCGTCACCCACCTCATCTTTGACATGGACGGACTTCTTCTGGATACTGAACGGCTGTATTCAGTGGTGTTTCAAGAAATATGTAATCGCTATGACAAGAAATACAGCTGGGATGTAAAGTCCCTGGTTATGGGTAAGAAGGCATTAGAGGCGGCACAGATTATAATAGACGTCTTGCAGCTCCCGATGTCCAAAGAGGAGCTGGTGGAAGAAAGCCAAACGAAGTTAAAGGAAGTGTTCCCCACGGCTGCGCTCATGCCAGGGGCGGAGAAACTCATCATCCACCTGCGGAAACATGGCATCCCCTTTGCACTGGCCACCAGCTCGGGGTCCGCGTCGTTCGATATGAAGACAAGCCGCCACAAGGAGTTCTTCAGCTTGTTTTCCCACATTGTGCTGGGAGATGACCCCGAAGTGCAGCATGGCAAGCCAGACCCGGACATCTTCCTAGCTTGTGCCAAGAGGTTCTCTCCCCCTCCTGCTATGGAGAAGTGCCTTGTC...,1


In [16]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
class DNADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Sequence
        self.labels = dataframe.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        dna_sequence = str(self.text[index])
        inputs = self.tokenizer(dna_sequence, return_tensors='pt', max_length=self.max_len, padding='max_length', truncation=True)
        labels = self.labels[index]
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [18]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50
)

In [19]:
train_dataset = DNADataset(train_df, tokenizer, max_len=512)
val_dataset = DNADataset(val_df, tokenizer, max_len=512)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [20]:
trainer.train()
results = trainer.evaluate()
print(results)

KeyError: 132